In [ ]:
import tensorflow
import imgaug as ia
from imgaug import augmenters as iaa
import imageio
import numpy as np
import skimage
import cv2
from PIL import Image
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join

## Input parameters

In [ ]:
images_path = "./images/cvc/training/images/"
labels_path = "./images/cvc/training/labels/"
image_out_path = "./images/cvc/training_augmented/images/"
label_out_path = "./images/cvc/training_augmented/labels/"
show_on_image = "./images/cvc/training_augmented/overlay/"
n_classes = 1+6
aug_num = 10

## Program

In [ ]:
image_files = [f for f in listdir(images_path) if isfile(join(images_path, f))]
label_files = [f for f in listdir(labels_path) if isfile(join(labels_path, f))]

if '.DS_Store' in image_files:
    image_files.remove('.DS_Store')
if '.DS_Store' in label_files:
    label_files.remove('.DS_Store')
    
image_files.sort()
label_files.sort()
assert(len(image_files) == len(label_files))
n_items = len(label_files)
print("Number of input images: ", n_items)

In [ ]:
# img_buffer = [None]*n_items*aug_num
# seg_buffer = [None]*n_items*aug_num
# overlay_buffer = [None]*n_items*aug_num

for i in range(n_items):
    
    print("Processing image ", i, ": ", image_files[i])
    image = cv2.imread(join(images_path, image_files[i]))
    label = cv2.imread(join(labels_path, label_files[i]))
    segmap = label.astype(np.int32)
    segmap = segmap[:,:,0]
    segmap = ia.SegmentationMapOnImage(segmap, shape=image.shape, nb_classes=n_classes)
    
    # transformation types
    seq = iaa.Sequential([
    iaa.Fliplr(0.5), 
    iaa.Flipud(0.5),      
    iaa.Affine(rotate=[90, 180, 270], 
               scale={"x": (0.75, 1.25), "y": (0.75, 1.25)}, 
               translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, mode='constant', cval=255)], random_order=True)
    
    # transform into 20 images
    for j in range(aug_num):
        print("  Transformation ", j)
        seq_det = seq.to_deterministic()
        img = seq_det.augment_image(image)
        seg_out = seq_det.augment_segmentation_maps([segmap])[0].get_arr_int()
        seg_overlay = seq_det.augment_segmentation_maps([segmap])[0].draw_on_image(img)
        
        image_out = image_out_path + image_files[i][:-4] + '_' + str(j) + '.jpg'
        label_out = label_out_path + image_files[i][:-4] + '_' + str(j) + '.png'
        overlay_out = show_on_image + image_files[i][:-4] + '_' + str(j) + '.png'
        
        # save images
        im = Image.fromarray(img.astype(np.uint8))
#         img_buffer[i*aug_num + j] = im
        im.save(image_out)
        
        # save segmentations
        sg = Image.fromarray(seg_out.astype(np.uint8))
#         seg_buffer[i*aug_num + j] = sg
        sg.save(label_out)
        
        # save overlay for verification
        sv = Image.fromarray(seg_overlay.astype(np.uint8))
#         overlay_buffer[i*aug_num + j] = sv
        sv.save(overlay_out)

In [ ]:


for i in range(n_items):
    
    print("Saving image ", i, ": ", image_files[i])

    # transform into 20 images
    for j in range(aug_num):
        print("Saving  Transformation ", j)
        
        
        image_out = image_out_path + image_files[i][:-4] + '_' + str(j) + '.jpg'
        label_out = label_out_path + image_files[i][:-4] + '_' + str(j) + '.png'
        overlay_out = show_on_image + image_files[i][:-4] + '_' + str(j) + '.png'
        
        # save images
        im = img_buffer[i*aug_num + j]
        im.save(image_out)
        
        # save segmentations
        sg = seg_buffer[i*aug_num + j]
        sg.save(label_out)
        
        # save overlay for verification
        sv = overlay_buffer[i*aug_num + j]
        sv.save(overlay_out)